# Sentiment Analysis with Pyspark

One of the tools I'm deeply interested, but haven't had many chance to explore is Apache Spark. Most of the time, Pandas and Scikit-Learn is enough to handle the size of data I'm trying to build a model on. But that also means that I haven't had chance to deal with petabytes of data yet, and I want to be prepared for the case I'm faced with real big data.

I have tried some basic data manipulation with Pyspark before, but only to a very basic level. I want to learn more and be more comfortable in using Pyspark. This post is my endeavour to have a better understanding of Pyspark.

Python is great for data science modelling, thanks to its numerous modules and packages that help achieve data science goals. But what if the data you are dealing with cannot be fit into a single machine? Maybe you can implement careful sampling to do your analysis on a single machine, but with distributed computing framework like Pyspark, you can efficiently implement the task for large data sets.

Spark API is available in multiple programming languages (Scala, Java, Python and R). There are debates about how Spark performance varies depending on which language you run it on, but since the main language I have been using is Python, I will focus on Pyspark without going into too much detail of what language should I choose for Apache Spark.

Spark has three different data structures available through its APIs: RDD, Dataframe (this is different from Pandas dataframe), Dataset. For this post, I will work with Dataframe, and corresponding machine learning library Sparkml. I first decided on the data structure I would like to use based on the advice from the post in Analytics Vidhya. "Dataframe is much faster than RDD because it has metadata (some information about data) associated with it, which allows Spark to optimize query plan." You can find more comprehensive introduction from the original post.
https://www.analyticsvidhya.com/blog/2016/09/comprehensive-introduction-to-apache-spark-rdds-dataframes-using-pyspark/

And there's also an informative post on Databricks comparing different data structures of Apache Spark: "A Tale of Three Apache Spark APIs: RDDs, DataFrames, and Datasets".
https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

Then I figured out that I need to use Sparkml instead Sparkmllib, if I want to deal with dataframe. Sparkmllib is used with RDD, while Sparkml supports dataframe.

One more thing to note is that I will work in local mode with my laptop. The local mode is often used for prototyping, development, debugging, and testing. However, as Spark's local mode is fully compatible with the cluster mode, codes written locally can be run on a cluster with just a few additional steps.

In order to use Pyspark in Jupyter Notebook, you should either configure Pyspark driver or use a package called Findspark to make a Spark Context available in your Jupyter Notebook. You can easily install Findspark by "pip install findspark" in your command line. Let's first load some of the basic dependencies we need.

In [1]:
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

First step in any Apache programming is to create a SparkContext. SparkContext is needed when we want to execute operations in a cluster. SparkContext tells Spark how and where to access a cluster. It is first step to connect with Apache Cluster. 

In [2]:
from pyspark import SparkConf, SparkContext
import warnings

from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc


try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    
    spark = SparkSession.builder.appName("test_import").getOrCreate()
    sc = spark.sparkContext
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
    
except ValueError:
    warnings.warn("SparkContext already exists in this scope")
    
    
    
ssc = StreamingContext(sc, 2 )
ssc.checkpoint( "/Users/anujchaudhari/Desktop/256/project/samples/twitter_streaming/checkpoint")

socket_stream = ssc.socketTextStream("192.168.0.100", 5555)

lines = socket_stream.window(2 )

Just created a SparkContext


In [3]:
sc.master

'local[*]'

The dataset I'll use for this post is annotated Tweets from "Sentiment140". It originated from a Stanford research project, and I used this dataset for my previous series of Twitter sentiment analysis. Since I already cleaned the tweets during the process of my previous project, I will use pre-cleaned tweets. If you want to know more in detail about the cleaning process I took, you can check my previous post: "Another Twitter sentiment analysis with Python-Part 2"
https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-2-333514854913

In [4]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('clean_tweet.csv')

In [5]:
type(df)

pyspark.sql.dataframe.DataFrame

In [6]:
df.show(5)

+---+--------------------+------+
|_c0|                text|target|
+---+--------------------+------+
|  0|awww that s a bum...|     0|
|  1|is upset that he ...|     0|
|  2|i dived many time...|     0|
|  3|my whole body fee...|     0|
|  4|no it s not behav...|     0|
+---+--------------------+------+
only showing top 5 rows



In [7]:
df = df.dropna()

In [8]:
df.count()

1596753

After successfully loading the data as Spark Dataframe, we can take a peek at the data by calling .show(), which is equivalent to Pandas .head(). After dropping NA, we have a bit less than 1.6 million Tweets. I will split this in three parts; training, validation, test. Since I have around 1.6 million entries, 1% each for validation and test set will be enough to test the models.

In [9]:
(train_set, val_set, test_set) = df.randomSplit([0.01, 0.01, 0.98], seed = 2000)

In [10]:
print(test_set.head(5))

[Row(_c0=0, text='awww that s a bummer you shoulda got david carr of third day to do it d', target=0), Row(_c0=1, text='is upset that he can t update his facebook by texting it and might cry as a result school today also blah', target=0), Row(_c0=2, text='i dived many times for the ball managed to save the rest go out of bounds', target=0), Row(_c0=3, text='my whole body feels itchy and like its on fire', target=0), Row(_c0=4, text='no it s not behaving at all i m mad why am i here because i can t see you all over there', target=0)]


## N-gram Implementation

In Scikit-Learn, n-gram implementation is fairly easy. You can define range of n-grams when you call TfIdf Vectorizer. But with Spark, it is a bit more complicated. It does not automatically combine features from different n-grams, so I had to use VectorAssembler in the pipeline, to combine the features I get from each n-grams.

I first tried to extract around 16,000 features from unigram, bigram, trigram. This means I will get around 48,000 features in total. Then I implemented Chi Squared feature selection to reduce the features back to 16,000 in total.

In [11]:
from pyspark.ml.feature import NGram, VectorAssembler
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def build_ngrams_wocs(inputCol=["text","target"], n=3):
    tokenizer = [Tokenizer(inputCol="text", outputCol="words")]
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    cv = [
        CountVectorizer(vocabSize=5460,inputCol="{0}_grams".format(i),
            outputCol="{0}_tf".format(i))
        for i in range(1, n + 1)
    ]
    idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

    assembler = [VectorAssembler(
        inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
        outputCol="features"
    )]
    label_stringIdx = [StringIndexer(inputCol = "target", outputCol = "label")]
    lr = [LogisticRegression(maxIter=100)]
    return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+lr)

In [12]:
%%time

trigramwocs_pipelineFit = build_ngrams_wocs().fit(train_set)
predictions_wocs = trigramwocs_pipelineFit.transform(val_set)
accuracy_wocs = predictions_wocs.filter(predictions_wocs.label == predictions_wocs.prediction).count() / float(val_set.count())
#roc_auc_wocs = evaluator.evaluate(predictions_wocs)

# print accuracy, roc_auc
print("Accuracy Score: {0:.4f}".format(accuracy_wocs))


Accuracy Score: 0.6883
CPU times: user 189 ms, sys: 41.2 ms, total: 230 ms
Wall time: 50.5 s


In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

In [14]:
# test_predictions = trigramwocs_pipelineFit.transform(test_set)
# test_accuracy = test_predictions.filter(test_predictions.label == test_predictions.prediction).count() / float(test_set.count())
# #test_roc_auc = evaluator.evaluate(test_predictions)

# # print accuracy, roc_auc
# print("Accuracy Score: {0:.4f}".format(test_accuracy))
# #print("ROC-AUC: {0:.4f}".format(test_roc_auc))

This has given me almost same result, marginally lower, but the difference is in the fourth digit. Considering it takes only 8 mins without ChiSqSelector, I choose the model without ChiSqSelector.

And finally, let's try this model on the final test set.

Final test accuracy is 81.22%.

In [15]:
import time
from pyspark.sql import Row
from pyspark.sql import SparkSession


def tweet_cleaner_updated(text):
    print(text)
    return text

def process(rdd):
    print(rdd)    
    try:        
        spark = SparkSession.builder.appName("test_import").getOrCreate()
        
        tweet = rdd.collect()
        if len(tweet) != 0:
            tweet = list(tweet[0])
        else:
            tweet = []

        rows = []
        for i in range(len(tweet)):
            rows.append(Row(_c0=i,text=tweet[i],target=0))

        if len(rows) == 0:
            rows.append(Row(_c0=1,text="EMPTY TWEET",target=0))
            
        df = spark.createDataFrame(rows)
        df.registerTempTable("tweets")
        
    except Exception as e: 
        print(e)
    
lines = lines.map(lambda x: x.lower());
lines = lines.map(lambda x: x.replace(" rt " , " "))
lines = lines.map(lambda x: x.replace("\n" , " "))
lines = lines.reduce(lambda x,y : x + y)
lines = lines.map(lambda x: x.split(" $$$$$$ "))
lines = lines.map(lambda x: tweet_cleaner_updated(x))

lines.foreachRDD(lambda rdd: process(rdd))



In [16]:
ssc.start()


In [17]:
import redis

config = {
    'host' : 'localhost',
    'port' : 6379,
    'db' : 0
}

redis_object = redis.StrictRedis(**config)

channel = "tweet_prediction"

In [18]:
import time
count = 0
aaaaa = 0


time.sleep( 2 )

while count < 500:
    
    top_10_tweets = sqlContext.sql( 'Select * from tweets' )
    print("%%%%%%%%%%%%%%%%% BLOCK " + str(count) + " %%%%%%%%%%%%%%%%\n")    
    aaaaa = trigramwocs_pipelineFit.transform(top_10_tweets)
    #print(aaaaa)
    predicted_tweets = aaaaa.collect()
    
    for tweet in predicted_tweets:
        print("########################")
        print(tweet.text)
        print(tweet.prediction)
        print("########################\n")    

        message_body = '{"text": "'+ tweet.words[0] +'", "sentiment":"' + str(tweet.prediction) + '"}'
        message = '{message_body}'.format(**locals()).encode('UTF-8')
        redis_object.publish(channel, message)

    
    count = count + 1

PythonRDD[229] at RDD at PythonRDD.scala:48
%%%%%%%%%%%%%%%%% BLOCK 0 %%%%%%%%%%%%%%%%

PythonRDD[245] at RDD at PythonRDD.scala:48
########################
EMPTY TWEET
0.0
########################

%%%%%%%%%%%%%%%%% BLOCK 1 %%%%%%%%%%%%%%%%

########################
1 :  #rt @funder: @funder: mitch mcconnell refuses to hold a vote on a bill to protect robert mueller from being fire… https://t.co/dbuvwm65r6
1.0
########################

########################
2 :  @prisonplanet: apologizes for saying she would have voted for the sitting president.the sheer level of intellectual cowardice here is…
1.0
########################

########################
3 :  double standard: hillary lies through her teeth pay no price but flynn attacked ferociously  https://t.co/x5hcyxbnaf
0.0
########################

########################
4 :  who is this trump you speak of? i only know of lord satsuma
1.0
########################

########################
5 :  @ericuman: at least 4 people say they

########################
out of these 457 days, he has spent a total of 149 of them at a trump-own…
0.0
########################

########################
32 :  @pollingtrump: are you proud that @realdonaldtrump donald trump is your president?
1.0
########################

########################
33 :  @johnwdean: sean hannity's biggest conflict of interest is not unlike trump’s: inability to tell the truth. he will make a dandy witness…
0.0
########################

########################
34 :  @horwitzpolitics: coming from one of the longest line of jews here, i'm disgusted. i wanted to make known that this shit is so blatant n…
0.0
########################

########################
35 :  @fabulous_he: shania twain: i would've voted for trump the gays:https://t.co/3mlw19cr28
1.0
########################

########################
36 :  @dloesch: 2018: when the rage mob demands apologies for diverse thought. https://t.co/m2gbhmhl7c
0.0
########################

#####################

########################
https://t.co/czdbwfcvok
1.0
########################

########################
44 :  @jordanuhl: last night, a white gunman killed four innocent, unarmed people of color with an assault rifle in a tennessee waffle house.…
0.0
########################

########################
45 :  @mikandynothem: jack johnson, resented after defeating white boxer in 1910 that sparked race riots, was later convicted by white jury on…
1.0
########################

########################
46 :  @businessinsider: donald trump has been a public figure since the '80s — here's how he has changed since then https://t.co/mkjcoemboq
0.0
########################

########################
47 :  @john_kissmybot: congress the 2018 mid term elections are right around the corner 👉i suggest you get on board the trump train and pass…
0.0
########################

########################
48 :  john oliver compares donald trump administration to 'sharknado' - deadline https://t.co/wahuw2nzrv
0.

########################
new report reveals trump fears money laundering will come back an…
1.0
########################

########################
64 :  @politvidchannel: breaking: michael bloomberg announces on "face the nation" he will give $4.5 mill to pay for remaining us portion of p…
0.0
########################

########################
65 :  @lisasmith4680: i normally don't bring up politics at the cafe,but today i took a survey. living in lib nj, i was pleasantly surprised &amp;…
1.0
########################

########################
66 :  @dbongino: stupid move. now your weakness has alienated everyone. liberals will hate you anyway because they hate everything, themselves…
0.0
########################

########################
67 :  @moruf88 @kanyewest @realdonaldtrump what’s scary for me is being lied to and thinking it’s the truth. trump is fun… https://t.co/fsjymbwxnu
0.0
########################

########################
68 :  @ani_nomso: if ceec wins this money, forget 

PythonRDD[458] at RDD at PythonRDD.scala:48
########################
th…
1.0
########################

########################
71 :  lowkey want to make a side account where i pretend to like trump and praise him because maybe he’ll pay me for it b… https://t.co/mctp6w9qwk
1.0
########################

########################
72 :  watch: macron struggles to explain support collapse as riots plunge france into chaos  https://t.co/lu1gqnfmn1
1.0
########################

########################
73 :  @billboard: shania twain would have voted for donald trump 'even though he was offensive' https://t.co/7lu0opkhqf https://t.co/hacnvu9d5j
0.0
########################

########################
74 :  @sam_ezeh: recall sir @potus @realdonaldtrump that the man you're about to receive in the coming days is a mass murderer, anti-jew, anti…
1.0
########################

########################
75 :  trump slams ‘sleepy eyes’ chuck todd for criticizing nkorea peace efforts https://t.co/upg77zc

...

########################
no, he really doesn’t. like his bff trump, he is one giant conflict of int…
0.0
########################

########################
1500 :  black activist says trump policies unlike obama’s create jobs https://t.co/rxm3noolat
0.0
########################

########################
1501 :  @amy_siskind: it’s tempting to laugh off trump’s heinous tweets and say, that’s just who he is. fight that urge.  we cannot normalize an…
1.0
########################

########################
1502 :  @lorettafaucher: @rolandscahill and she never has a smile like that when she's around trump.
1.0
########################

########################
1503 :  @kevinjohnson510: denzel washington: with trump we avoided war with russia and orwellian police state https://t.co/mz99wbvavl
1.0
########################

########################
1504 :  @stephenmilier: donald trump has now destroyed 17 presidential candidates and 2 fbi directors.
0.0
########################

################

########################
no, he really doesn’t. like his bff trump, he is one giant conflict of int…
0.0
########################

########################
1500 :  black activist says trump policies unlike obama’s create jobs https://t.co/rxm3noolat
0.0
########################

########################
1501 :  @amy_siskind: it’s tempting to laugh off trump’s heinous tweets and say, that’s just who he is. fight that urge.  we cannot normalize an…
1.0
########################

########################
1502 :  @lorettafaucher: @rolandscahill and she never has a smile like that when she's around trump.
1.0
########################

########################
1503 :  @kevinjohnson510: denzel washington: with trump we avoided war with russia and orwellian police state https://t.co/mz99wbvavl
1.0
########################

########################
1504 :  @stephenmilier: donald trump has now destroyed 17 presidential candidates and 2 fbi directors.
0.0
########################

################

########################
they know it.…
1.0
########################

########################
1529 :  @kcodunn: remember when the gop was using scare tactics about death panels in the obama care bill? well trump is 71 or 80, who can tell?…
1.0
########################

########################
1530 :  @johnwdean: sean hannity's biggest conflict of interest is not unlike trump’s: inability to tell the truth. he will make a dandy witness…
0.0
########################

########################
1531 :  @dearauntcrabby: obama is more admired than trump in every country except russia: poll https://t.co/tvyqjowrg2
0.0
########################

########################
1532 :  @stopthenutjob: pruitt &amp; trump are climate criminalswho should be tried in international courtfor crimes against humanitylies &amp; denie…
1.0
########################

########################
1533 :  @proudresister: neo-nazis rally in georgia.trump has not condemned this.
0.0
########################

%%%%%%%%%%%%

KeyboardInterrupt: 

PythonRDD[4151] at RDD at PythonRDD.scala:48
PythonRDD[4163] at RDD at PythonRDD.scala:48
PythonRDD[4175] at RDD at PythonRDD.scala:48


In [19]:
ssc.stop()